In [2]:
import requests
import json
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from datetime import datetime
from datetime import timedelta
from fabric import Connection
import psycopg2
import gmaps
import pandas.io.sql as sqlio
import getpass
from io import StringIO
import paramiko
import sys
from time import sleep
from tqdm import tqdm

sys.path.insert(0, "/home/jovyan/watexr/PROGNOS/")
from prognos_tools.gce_light import gce_api as gce

key = getpass.getpass('mobiserver password: ')
cloudKey = getpass.getpass('vault password: ')
#Querying necessary tokens
def query(query,fetch=True):
    with psycopg2.connect(user='jose-luis', host='mobiserver.niva.no', port=5432, database='vault',password=key) as db:
        with db.cursor() as cursor :
            cursor.execute(query)
            if fetch:
                result = sqlio.read_sql_query(query, db)
                return result
            
# def insertFile(alias,filename):
#     with psycopg2.connect(user='jose-luis', host='mobiserver.niva.no', port=5432, database='vault',password=key) as db:
#         with db.cursor() as cursor :
#             cursor.execute('drop table if exists dummy;')
#             cursor.execute('''create temp table dummy (sometext bytea);''')
#             with open(filename,'rb') as f:
# #                 cursor.copy_from(f,'dummy')
#                 cursor.execute('''insert into dummy  select %s::bytea;''',(psycopg2.Binary(f.read()),) )
#             cursor.execute('''select niva.insertToken('{}',(select * from dummy),'');'''.format(alias));

gmapsKey = query('''select niva.getToken('gmaps','{}');'''.format(cloudKey)).iloc[0,0]
sshKey = query('''select niva.getToken('geonorway_ssh_key','{}');'''.format(cloudKey)).iloc[0,0]
nveKey = query('''select niva.getToken('nveKey','{}');'''.format(cloudKey)).iloc[0,0]
cloudKey = json.loads(query('''select niva.getToken('gce_access','{}');'''.format(cloudKey)).iloc[0,0])
gmaps.configure(gmapsKey)
not_really_a_file = StringIO(sshKey)
private_key = paramiko.RSAKey.from_private_key(not_really_a_file)
del key,sshKey

mobiserver password:  ·······
vault password:  ···············


In [3]:
#Check status of instance
properties = {'project'      : 'nivacatchment',
             'zone'         : 'europe-north1-a',
             'instanceType' : "n1-standard-4",
             'instanceName' : "dtm10",
             'username'     : "jose-luis",
             }

cloud = gce(properties, cloudKey)
del cloudKey

In [4]:
#Getting instance info
cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}'''
info = cloud.get('custom')
display(info['status'])
#If instance is stopped, start it
if info['status'] != 'RUNNING':
    cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}/start'''
    info = cloud.post('custom')
    display(info['status'])
    cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}'''
    info = cloud.get('custom')
    while info['status'] != 'RUNNING':
           sleep(2)
           info = cloud.get('custom')
          
geonorway = info['networkInterfaces'][0]['accessConfigs'][0]['natIP']    
    
display(geonorway)
config =  {'host' : geonorway, 'user': 'jose-luis', 'connect_kwargs': {'pkey': private_key } }

'RUNNING'

'35.228.213.48'

# Getting nve data

In [5]:
#Creating header to query nve data
headers={'accept': 'application/json',
         'X-API-Key': '{}'.format(nveKey)}

# r = requests.get('https://hydapi.nve.no/api/v1/Parameters', headers=headers)
# display(json.loads(r.text))                  

In [6]:
parameter = 1001 #Discharge
r = requests.get('https://hydapi.nve.no/api/v1/Stations', headers=headers)
data = json.loads(r.text)
stations_with_q = [{'stationName' : i['stationName'],
                   'stationId'    : i['stationId'],
                   'latitude'     : i['latitude'],
                   'longitude'    : i['longitude'],
                   'parameters'   : i['seriesList']
                   }
                  for i in data['data'] if parameter in [j['parameter'] for j in i['seriesList']]]

allStations = [{'stationName' : i['stationName'],
                   'stationId'    : i['stationId'],
                   'latitude'     : i['latitude'],
                   'longitude'    : i['longitude'],
                   'parameters'   : i['seriesList']
                   }
                  for i in data['data'] ]


In [7]:
with open('dummy.txt','w') as f:
    for ii in stations_with_q : 
        f.write(json.dumps(ii))
        f.write('\n\n')

In [8]:
len(stations_with_q)#data

604

In [10]:
schema = 'nvedischarge'
def query(query,fetch=True):
    with psycopg2.connect(user='jose-luis', host=geonorway, port=5432, database='geonorway') as db:
        cursor = db.cursor()
        cursor.execute(query)
        if fetch:
            result = sqlio.read_sql_query(query, db)
            return result
        
query('''drop schema if exists {0} cascade;
create schema {0};'''.format(schema),fetch=False)

        
sql = '''create table {0}.stations (sid int generated always as identity,
stationName varchar(100) not null,
stationID varchar(50) unique not null,
latitude double precision not null,
longitude double precision not null,
unique(sid),
primary key (sid)
);

insert into {0}.stations (stationName,stationId,latitude,longitude) values {1};

create index {0}_stations_station_id_idx on {0}.stations(stationId);
alter table {0}.stations add column geom geometry(point,4326);
update {0}.stations
set geom = st_setsrid(st_makepoint(longitude,latitude),4326);
'''.format(schema,',\n'.join(["('" + i['stationName'].replace("'","''") + "',"
                     + "'" + i['stationId'] + "'" + ','
                     + str(i['latitude']) + ','
                     + str(i['longitude']) 
                     + ')'
                    for i in data['data'] if parameter in [j['parameter'] for j in i['seriesList']]
                    ]
                    )
          )

# with open('dummy.sql','w') as f:
#     f.write(sql)
query(sql,fetch=False)

In [11]:
#Storing metadata for all stations
sql = '''drop table if exists {0}.metadata;
create table {0}.metadata (mid int generated always as identity, sid int,  metadata json,
primary key(mid),
constraint nve_stations_k
foreign  key(sid)
references {0}.stations(sid)
);
'''.format(schema)
query(sql,fetch=False)

sids =  query('''select sid,stationId from {}.stations;'''.format(schema))

for ii,jj in tqdm(sids.iterrows()):
#     display(jj['sid'],jj['stationid'])
    r = requests.get('https://hydapi.nve.no/api/v1/Series?StationId={}'.format(jj['stationid']),headers=headers);
#     display('{} -> {}'.format(ii,r.status_code))
    sleep(0.1)
    query('''insert into {}.metadata(sid,metadata) values ({},'{}')'''.format(schema,jj['sid'],r.text.replace("'","''")),fetch=False)
# len(sids)

604it [02:57,  3.40it/s]


In [12]:
# display(query('''select json_array_elements(metadata #> '{data}') from nvedata.metadata limit 10;'''))
# a = query('''select json_array_elements(metadata -> 'data')->'stationId' from nvedata.metadata limit 10;''')
query('''drop table if exists {}.singledata;'''.format(schema),fetch=False)
query('''create table {0}.singledata as select sid, json_array_elements(metadata -> 'data') as jdoc from {0}.metadata;'''.format(schema),fetch=False)
query('''alter table {}.singledata add column id int generated always as identity;'''.format(schema),fetch=False)
query('''alter table {}.singledata add primary key(id);'''.format(schema),fetch=False)
query('''alter table {0}.singledata add constraint nve_stations_k foreign key(sid) references {0}.stations(sid);'''.format(schema) ,fetch=False)
query('''alter table {}.singledata alter column jdoc set data type jsonb using jdoc::jsonb;'''.format(schema),fetch=False)
a = query('''select * from {}.singledata;'''.format(schema))
display(a)

,sid,jdoc,id
0,1,"{'masl': 177, 'unit': 'm', 'serieTo': None, 'l...",1
1,1,"{'masl': 177, 'unit': 'm³/s', 'serieTo': '2005...",2
2,1,"{'masl': 177, 'unit': '°C', 'serieTo': '2005-0...",3
3,1,"{'masl': 177, 'unit': '°C', 'serieTo': None, '...",4
4,2,"{'masl': 134, 'unit': 'm', 'serieTo': None, 'l...",5
...,...,...,...
1841,603,"{'masl': 462, 'unit': 'm³/s', 'serieTo': None,...",1842
1842,603,"{'masl': 462, 'unit': '°C', 'serieTo': None, '...",1843
1843,604,"{'masl': 563, 'unit': 'm', 'serieTo': None, 'l...",1844
1844,604,"{'masl': 563, 'unit': 'm³/s', 'serieTo': None,...",1845


In [13]:
query('''drop table if exists {0}.dummy'''.format(schema),fetch=False)
query('''create table {0}.dummy as select distinct a.sid,
(a.jdoc->>'latitude')::float as latitude, 
(a.jdoc->>'longitude')::float as longitude
,b.stationName, b.stationId, jsonb_array_elements(a.jdoc -> 'resolutionList') -> 'resTime' as resTime, 
jsonb_array_elements(a.jdoc -> 'resolutionList') -> 'dataFromTime' as fromTime,
jsonb_array_elements(a.jdoc -> 'resolutionList') -> 'dataToTime' as toTime,
a.jdoc -> 'parameter' as parameter,
a.jdoc -> 'versionNo' as version
from {0}.singledata as a join {0}.stations as b on b.sid=a.sid 
where jdoc @>'{{"parameter" : {1} }}'
'''.format(schema,parameter),fetch=False)
water_temp = query('''select * from {}.dummy where resTime='0' '''.format(schema))
display(water_temp)

,sid,latitude,longitude,stationname,stationid,restime,fromtime,totime,parameter,version
0,1,59.92993,11.40768,Berg,1.198.0,0,1991-06-24T09:00:00Z,2005-07-22T12:00:00Z,1001,1
1,2,59.92038,11.53180,Lierelv,1.200.0,0,2010-09-15T15:00:00Z,2021-04-14T05:00:00Z,1001,1
2,3,59.48267,11.65056,Ørje,1.48.0,0,1963-09-01T11:00:00Z,2021-04-11T11:00:00Z,1001,1
3,4,59.14763,11.55638,Brekke,1.49.0,0,2020-12-13T11:00:00Z,2021-04-11T11:00:00Z,1001,1
4,5,59.12735,11.44301,Tistedalsfoss,1.50.0,0,2020-12-13T11:00:00Z,2021-04-11T11:00:00Z,1001,2
...,...,...,...,...,...,...,...,...,...,...
605,600,62.07259,6.45240,Sleddalen,97.5.0,0,1997-12-16T14:00:00Z,2021-04-14T07:00:00Z,1001,1
606,601,62.06818,6.93163,Øye ndf.,98.4.0,0,1990-10-02T10:30:00Z,2021-04-14T07:00:00Z,1001,1
607,602,62.21919,7.42720,Onilsavatn,99.1.0,0,1943-10-20T11:00:00Z,1975-08-18T11:00:00Z,1001,2
608,603,62.19769,7.49543,Rødøla,99.17.0,0,1981-05-01T11:00:00Z,2021-02-28T23:00:00Z,1001,1


In [14]:
display(water_temp.loc[water_temp['version']==2])

,sid,latitude,longitude,stationname,stationid,restime,fromtime,totime,parameter,version
4,5,59.12735,11.44301,Tistedalsfoss,1.50.0,0,2020-12-13T11:00:00Z,2021-04-11T11:00:00Z,1001,2
8,9,62.28049,8.11780,Ulvåa v/Storhølen,103.1.0,0,1971-05-07T11:00:00Z,2021-04-14T07:00:00Z,1001,2
12,13,62.27660,8.15311,Rauma v/Stuguflåten,103.3.0,0,1971-05-08T11:00:00Z,2021-04-14T07:00:00Z,1001,2
18,19,62.79320,7.72607,Osenelv v/Øren,105.1.0,0,1923-05-05T11:00:00Z,2021-04-14T07:00:00Z,1001,2
23,23,62.64874,8.69409,Driva v/Elverhøy bru,109.42.0,0,2008-01-30T12:00:00Z,2020-06-04T17:00:00Z,1001,2
30,29,62.79663,8.78269,Toåa v/Talgøyfoss,111.5.0,0,1944-01-01T11:00:00Z,2021-04-13T19:00:00Z,1001,2
32,31,62.98827,9.14641,Follsjø,112.13.0,0,1968-09-01T11:00:00Z,2016-01-05T23:00:00Z,1001,2
33,32,62.93264,9.12840,Gråsjø,112.14.0,0,1968-10-24T11:00:00Z,2016-01-05T23:00:00Z,1001,2
40,39,60.04321,9.63403,Kråkefjord ndf.,12.113.0,0,1976-01-01T11:00:00Z,2021-04-14T06:00:00Z,1001,2
64,62,60.22176,10.36211,Kistefoss,12.228.0,0,2018-12-31T23:00:00Z,2021-04-14T05:30:00Z,1001,2


In [15]:
query('''alter table {0}.dummy add column geom geography(point,4326);
update {0}.dummy set geom = ST_SetSRID(ST_MakePoint(longitude, latitude),4326);'''.format(schema),fetch=False)

In [16]:
a = query('''select a.stationname,a.stationid,a.fromtime,a.totime, st_distance( a.geom, ST_SetSRID(ST_MakePoint(7.5281230, 59.0976410),4326) ) / 1000 as distance from 
{}.dummy as a where restime = '0' order by st_distance(a.geom,ST_SetSRID(ST_MakePoint(7.5281230, 59.0976410),4326) );'''.format(schema))
a.to_csv('nve_temp.csv')

## Displaying stations with discharge

In [17]:
fig = gmaps.figure(map_type="TERRAIN")
info_box_template = """
    <dl>
    <font color="black">
    <dt>Name</dt><dd>{stationname}</dd>
    <dt>Id</dt><dd>{stationid}</sup></dd>
    <dt>fromTime</dt><dd>{fromtime}</sup></dd>
    <dt>toTime</dt><dd>{totime}</sup></dd>
    <dt>Long</dt><dd>{longitude:.8f}</dd>
    <dt>Lat</dt><dd>{latitude:.8f}</dd>
    </font>
    </dl>
    """  
locations = [(i,j) for i,j in zip(water_temp['latitude'],water_temp['longitude'])]
station_info = [info_box_template.format(**i[1]) for i in water_temp.iterrows()]                                                 
marker_layer = gmaps.marker_layer(locations, label=water_temp['stationname'].values, info_box_content=station_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
#Redefining query()
def query(query,fetch=True):
    with psycopg2.connect(user='jose-luis', host=geonorway, port=5432, database='geonorway') as db:
        cursor = db.cursor()
        cursor.execute(query)
        if fetch:
            result = sqlio.read_sql_query(query, db)
            return result

# sids = list(water_temp['stationid'].values)
# r = requests.get('https://hydapi.nve.no/api/v1/Series?StationId={}'.format(sid),headers=headers);
# display(r)
# info = json.loads(r.text)
# display(info)

# schema = 'nvedata'
# query('drop schema if exists {} cascade;'.format(schema),fetch=False)
# query('create schema {};'.format(schema),fetch=False)

# #Creating tables with nve ids
# query('''drop table if exists {}.nveStations cascade;'''.format(schema),fetch=False)
# query('''create table {}.nveStations(id serial unique, nveId varchar(50),primary key(id));'''.format(schema),fetch = False)
# query('''insert into {}.nveStations (nveId) values {} '''.format(schema,','.join(["('" + i + "')" for i in sids])),fetch=False)

In [19]:
table_name = 'discharge'

sql = '''drop table if exists {0}.{1};
create table {0}.{1}(sid int,fecha timestamptz,{1} double precision,correction double precision, quality integer, version integer, unique(fecha,{1},correction,quality,version),
constraint idk
foreign key(sid)
references {0}.stations(sid));
'''.format(schema,table_name)

with open('q.sql','w') as f:
    f.write(sql)
with Connection(**config) as c:
    c.run('rm -f q.sql')
    c.put('q.sql')
    c.run('psql -q -d geonorway -f q.sql')
    c.run('rm -f q.sql')
    
def raw_to_string(i,currentId,version):
    return f'''({str(currentId)},
    '{datetime.strptime(i['time'],('%Y-%m-%dT%H:%M:%SZ')).strftime('%Y-%m-%d %H:%M:%S') + '+00' }',
    {i['value'] or 'Null'},
    {i['correction']},
    {i['quality']},
    {str(version)}
    )'''


def date_range(start, end, intv):    
    start = datetime.strptime(start,"%Y-%m-%d")
    end = datetime.strptime(end,"%Y-%m-%d")
    diff = (end  - start ) / intv
    for i in range(intv):
        yield (start + diff * i)#.strftime("%Y-%m-%d")
    yield end#.strftime("%Y-%m-%d")
    

def splitter(length,base=1,power=2):
    while True:
        for i in range(length):
            yield base * power**i
    


for i,j in water_temp.iterrows():
    display('Downloading data for {}'.format(str(j['stationid'])));
    parameter = j['parameter']
    restime = j['restime']
    referencetime = '{}/{}'.format(j['fromtime'].split('T')[0],j['totime'].split('T')[0])
    currentId = j['stationid']
    version = j['version']
    
    #First attempt at downloading the data
    r = requests.get('https://hydapi.nve.no/api/v1/Observations',headers=headers,params={'StationId': currentId, 
                                                                                             'Parameter': parameter, 
                                                                                             'ResolutionTime' : j['restime'],
                                                                                             'ReferenceTime' : referencetime,
                                                                                             'VersionNumber' : version})
    
    if r.status_code == 200:
        raw_data = json.loads(r.text)['data'][0]['observations']
        if len(raw_data) > 0 :
            values = [raw_to_string(l,j['sid'],version) for l in raw_data if raw_to_string(l,j['sid'],version) is not None]
            with open('q.sql','w') as f:
                f.write('''insert into {}.{} values {} on conflict do nothing;'''.format(schema,table_name,','.join(values)))
                f.write('\n')
            with Connection(**config) as c:
                c.run('rm -f q.sql')
                c.put('q.sql')
                c.run('psql -q -d geonorway -f q.sql')
                c.run('rm -f q.sql')
    else :
        
        start_date = referencetime.split('/')[0] 
        num_chunks = splitter(12)
        next(num_chunks) #So we start at two and not at one
        while r.status_code != 200: #Keep on trying until the command succeeds. Probably not the best of ideas since this means all the edge cases need to be handled below
            if r.text.startswith('"The query gives more than 150000 observations'):  #Other conditions will potentially need to be added if some other errors are generated
                #Splitting the data
                #There might be too many data within the reference time so we split it in chunks if that happens
                #The chunks  become increasingly smaller if the "too much data" error persits   
                splits=list(date_range(start_date,referencetime.split('/')[1],next(num_chunks)))
                intervals = [[i,j] for i,j in zip(splits[:-1],splits[1:])]
                intervals[1:] = [[i[0] ,i[1]] for i in intervals[1:]]
            for k in intervals:
                timechunk = '{}/{}'.format(k[0].strftime('%Y-%m-%d'),k[1].strftime('%Y-%m-%d'))
                display("Downloading period : {}".format(timechunk))
                r = requests.get('https://hydapi.nve.no/api/v1/Observations',headers=headers,params={'StationId': currentId, 
                                                                                                     'Parameter': parameter, 
                                                                                                     'ResolutionTime' : j['restime'],
                                                                                                     'ReferenceTime' : timechunk,
                                                                                                     'VersionNumber': version})
                if r.status_code == 200:
                    raw_data = json.loads(r.text)['data'][0]['observations']
                    if len(raw_data) > 0 :
                        values = [raw_to_string(l,j['sid'],version) for l in raw_data if raw_to_string(l,j['sid'],version) is not None]
                        with open('q.sql','w') as f:
                            f.write('''insert into {}.{} values {} on conflict do nothing;'''.format(schema,table_name,','.join(values)))
                            f.write('\n')
                        with Connection(**config) as c:
                            c.run('rm -f q.sql')
                            c.put('q.sql')
                            c.run('psql -q -d geonorway -f q.sql')
                            c.run('rm -f q.sql')
                else : 
#                     if k[1]!=k[0]:
                    display("Failed for the current interval, resetting starting point and number of chunks")
                    start_date = (k[0] - timedelta(days=1)).strftime('%Y-%m-%d')
                    break
                
                
#             display(r.text)
#                 sleep(0.25)
                
                      
        



#     display(json.loads(r.status))

#     splits=list(date_range(referencetime.split('/')[0],referencetime.split('/')[1],50))
#     intervals = [[i,j] for i,j in zip(splits[:-1],splits[1:])]
#     intervals[1:] = [[i[0] + timedelta(days=1),i[1]] for i in intervals[1:]]

    
#     currentId = j['stationid']
    
# #     #Remember that nve time information is given in zulu time
#     for k in intervals:
#         timechunk = '{}/{}'.format(k[0].strftime('%Y-%m-%d'),k[1].strftime('%Y-%m-%d'))
# #         display("Downloading period : {}".format(timechunk))
#         r = requests.get('https://hydapi.nve.no/api/v1/Observations',headers=headers,params={'StationId': currentId, 
#                                                                                              'Parameter': parameter, 
#                                                                                              'ResolutionTime' : j['restime'],
#                                                                                              'ReferenceTime' : timechunk})
# #         display(r.text)
# #         sleep(0.25)
#         while (r.status_code != 200):
#             display(r.text)
#             sleep(2)
#             r = requests.get('https://hydapi.nve.no/api/v1/Observations',headers=headers,params={'StationId': currentId, 
#                                                                                              'Parameter': parameter, 
#                                                                                              'ResolutionTime' : j['restime'],
#                                                                                              'ReferenceTime' : timechunk})
        
#         raw_data = json.loads(r.text)['data'][0]['observations']
        
#         if len(raw_data) > 0 :
#             values = [raw_to_string(l,j['sid']) for l in raw_data if raw_to_string(l,j['sid']) is not None]
#             with open('q.sql','w') as f:
#                 f.write('''insert into {}.watertemp values {};'''.format(schema,','.join(values)))
#                 f.write('\n')
#             with Connection(**config) as c:
#                 c.run('rm -f q.sql')
#                 c.put('q.sql')
#                 c.run('psql -q -d geonorway -f q.sql')
#                 c.run('rm -f q.sql')

        
        
with open('q.sql','w') as f:
    f.write('drop index if exists {}_fecha_idx;'.format(schema))
    f.write('\n')
    f.write('create index {0}_fecha_idx on {0}.{1} using btree(fecha);'.format(schema,table_name))
    f.write('\n')
    f.write('create index {0}_id_idx on {0}.{1}(sid);'.format(schema,table_name))
    f.write('\n')
    f.write('create index {0}_version_idx on {0}.{1}(version);'.format(schema,table_name))
with Connection(**config) as c:
    c.run('rm -f q.sql')
    c.put('q.sql')
    c.run('psql -q -d geonorway -f q.sql')
    c.run('rm -f q.sql')
    
        
display('Done')

psql:q.sql:1: NOTICE:  table "discharge" does not exist, skipping


'Downloading data for 1.198.0'

'Downloading data for 1.200.0'

'Downloading data for 1.48.0'

'Downloading data for 1.49.0'

...

'Downloading period : 1981-05-01/2001-03-31'

'Downloading period : 2001-03-31/2021-02-28'

'Downloading data for 99.2.0'

psql:q.sql:1: NOTICE:  index "nvedischarge_fecha_idx" does not exist, skipping


'Done'

In [ ]:
#Displaying data for one station
a = query('''select * from {schema}.discharge as a join nveq.stations as b on b.sid=a.sid where b.stationid='123.20.0' order by fecha;'''.format(schema=schema))
a.set_index('fecha',inplace=True)
a['discharge'].plot(figsize=(20,4))

In [21]:
display(query('''select * from nveq.stations where stationid = '50.3.0';  '''))

,sid,stationname,stationid,latitude,longitude,geom
0,485,Eidfjordvatn,50.3.0,60.44957,7.09805,0101000020E6100000BF7D1D3867641C40A1A17F828B39...


In [27]:
display(query('''select * from nvedischarge.dummy order by (totime::text::timestamptz - fromtime::text::timestamptz) desc limit 10; '''))

,sid,latitude,longitude,stationname,stationid,restime,fromtime,totime,parameter,version,geom
0,94,63.38490,10.39023,Rate,123.20.0,0,1881-05-22T11:00:00Z,2020-12-17T10:00:00Z,1001,0,0101000020E61000000FD6FF39CCC724407E1D386744B1...
1,61,59.92897,9.96283,Geithus kraftverk,12.220.0,0,1887-04-15T11:00:00Z,2019-06-05T08:00:00Z,1001,0,0101000020E61000002237C30DF8EC2340897B2C7DE8F6...
2,61,59.92897,9.96283,Geithus kraftverk,12.220.0,1440,1887-04-15T11:00:00Z,2019-06-04T11:00:00Z,1001,0,0101000020E61000002237C30DF8EC2340897B2C7DE8F6...
3,514,60.62868,6.29254,Bulken (Vangsvatnet),62.5.0,0,1892-01-01T11:00:00Z,2021-04-14T07:00:00Z,1001,1,0101000020E61000005D16139B8F2B19401CD313967850...
4,514,60.62868,6.29254,Bulken (Vangsvatnet),62.5.0,1440,1892-01-01T11:00:00Z,2021-04-13T11:00:00Z,1001,1,0101000020E61000005D16139B8F2B19401CD313967850...
5,441,61.91996,11.94004,Femundsenden (Femunden),311.4.0,0,1896-01-01T11:00:00Z,2021-04-14T07:00:00Z,1001,1,0101000020E61000001ADD41EC4CE1274068D0D03FC1F5...
6,278,61.33339,10.27753,Losna,2.145.0,0,1896-01-01T11:00:00Z,2021-04-14T06:00:00Z,1001,1,0101000020E610000059518369188E24401F680586ACAA...
7,441,61.91996,11.94004,Femundsenden (Femunden),311.4.0,1440,1896-01-01T11:00:00Z,2021-04-13T11:00:00Z,1001,1,0101000020E61000001ADD41EC4CE1274068D0D03FC1F5...
8,278,61.33339,10.27753,Losna,2.145.0,1440,1896-01-01T11:00:00Z,2021-04-13T11:00:00Z,1001,1,0101000020E610000059518369188E24401F680586ACAA...
9,385,58.12024,7.52813,Kjølemo,22.4.0,0,1896-08-01T11:00:00Z,2021-04-14T06:00:00Z,1001,1,0101000020E61000005B25581CCE1C1E40E5D53906640F...


In [ ]:
display(query('''select '''))